In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

617

In [5]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Import the datetime module from the datetime library.
from datetime import datetime

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=4c3237a5d3810ea714b644930e53e5a7


In [36]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | namibe
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | san patricio
Processing Record 4 of Set 1 | saint-philippe
Processing Record 5 of Set 1 | bukama
Processing Record 6 of Set 1 | busselton
Processing Record 7 of Set 1 | osorno
Processing Record 8 of Set 1 | malsch
Processing Record 9 of Set 1 | nicoya
Processing Record 10 of Set 1 | bud
Processing Record 11 of Set 1 | mogadishu
Processing Record 12 of Set 1 | kaitangata
Processing Record 13 of Set 1 | odienne
Processing Record 14 of Set 1 | janikowo
Processing Record 15 of Set 1 | avarua
Processing Record 16 of Set 1 | hay river
Processing Record 17 of Set 1 | albany
Processing Record 18 of Set 1 | tlanalapan
Processing Record 19 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 20 of Set 1 | yantongshan
Processing Record 21 of Set 1 | qaanaaq
Processing Record 22 of Set 1 | puerto del rosario
Pro

In [37]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Namibe,AO,2021-04-27 23:21:17,-15.1961,12.1522,75.34,68,14,7.94
1,Rikitea,PF,2021-04-27 23:16:42,-23.1203,-134.9692,77.13,73,29,11.23
2,San Patricio,US,2021-04-27 23:21:18,28.0170,-97.5169,78.80,83,90,19.57
3,Saint-Philippe,RE,2021-04-27 23:21:18,-21.3585,55.7679,75.20,83,0,8.05
4,Bukama,CD,2021-04-27 23:21:18,-9.2000,25.8500,69.17,94,52,2.82
5,Busselton,AU,2021-04-27 23:21:19,-33.6500,115.3333,54.00,98,33,3.94
6,Osorno,CL,2021-04-27 23:21:19,-40.5667,-73.1500,60.80,84,90,12.66
7,Malsch,DE,2021-04-27 23:21:19,48.8833,8.3333,50.00,57,10,6.91
8,Nicoya,CR,2021-04-27 23:21:19,10.1483,-85.4520,87.80,45,40,14.97
9,Bud,US,2021-04-27 23:21:20,39.4470,-86.1758,78.80,31,1,11.50


In [38]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")